In [30]:
# !rm -rf /kaggle/working/*
!pip install datasets

In [13]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, AutoConfig
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, num_labels=3)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [15]:
import os
print(os.listdir("/kaggle/input/balanced-labeled-comments-dataset-int-csv"))
import pandas as pd

dframe = pd.read_csv("/kaggle/input/balanced-labeled-comments-dataset-int-csv/balanced_labeled_comments_dataset_int.csv")


# dframe['Classification'] = dframe['Classification'].map({"Other": 0, "Suggestion": 1})

# Split the data
train_df, temp_df = train_test_split(dframe, test_size=0.1, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Create DatasetDict
categories = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

categories

['balanced_labeled_comments_dataset_int.csv']


DatasetDict({
    train: Dataset({
        features: ['Comments', 'Classification'],
        num_rows: 5400
    })
    validation: Dataset({
        features: ['Comments', 'Classification'],
        num_rows: 300
    })
    test: Dataset({
        features: ['Comments', 'Classification'],
        num_rows: 300
    })
})

In [16]:
# Tokenize the datasets
def tokenize(batch):
    return tokenizer(batch["Comments"], padding=True, truncation=True, max_length=512)

categories_encoded = categories.map(tokenize, batched=True, batch_size=None)
BATCH_SIZE = 16

def prepare_tf_dataset(dataset):
    # Convert to tensorflow dataset
    tf_dataset = tf.data.Dataset.from_tensor_slices(({
        'input_ids': dataset['input_ids'],
        'attention_mask': dataset['attention_mask']
    }, dataset['Classification']))

    return (tf_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

train_dataset = prepare_tf_dataset(categories_encoded['train'])
val_dataset = prepare_tf_dataset(categories_encoded['validation'])
test_dataset = prepare_tf_dataset(categories_encoded['test'])

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [18]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_dataset,
    epochs=5
)

Epoch 1/5
338/338 [==============================] - 194s 529ms/step - loss: 0.2777 - accuracy: 0.8931
Epoch 2/5
338/338 [==============================] - 179s 528ms/step - loss: 0.1783 - accuracy: 0.9419
Epoch 3/5
338/338 [==============================] - 179s 528ms/step - loss: 0.1197 - accuracy: 0.9607
Epoch 4/5
338/338 [==============================] - 179s 529ms/step - loss: 0.0725 - accuracy: 0.9785
Epoch 5/5
338/338 [==============================] - 179s 529ms/step - loss: 0.0520 - accuracy: 0.9856


In [31]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy:.4f}")

# Save the model in TensorFlow SavedModel format
!mkdir "/kaggle/working/distilbert_suggestion_model"
tf.saved_model.save(model, "/kaggle/working/distilbert_suggestion_model")

19/19 [==============================] - 3s 165ms/step - loss: 0.2184 - accuracy: 0.9300
Test accuracy: 0.9300


In [32]:
# Zip the model directory
!zip -r /kaggle/working/distilbert_suggestion_model.zip /kaggle/working/distilbert_suggestion_model

  adding: kaggle/working/distilbert_suggestion_model/ (stored 0%)
  adding: kaggle/working/distilbert_suggestion_model/saved_model.pb (deflated 92%)
  adding: kaggle/working/distilbert_suggestion_model/variables/ (stored 0%)
  adding: kaggle/working/distilbert_suggestion_model/variables/variables.data-00000-of-00001 (deflated 24%)
  adding: kaggle/working/distilbert_suggestion_model/variables/variables.index (deflated 78%)
  adding: kaggle/working/distilbert_suggestion_model/fingerprint.pb (stored 0%)
  adding: kaggle/working/distilbert_suggestion_model/assets/ (stored 0%)


In [13]:
import tensorflow as tf
from transformers import AutoTokenizer
import numpy as np

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
loaded_model = tf.saved_model.load("/kaggle/working/distilbert_suggestion_model")

# Model inference function
infer = loaded_model.signatures["serving_default"]

# Comments for prediction
comments_list = [
    'misleading title click bait',
    'kindly post video about claude 35 sonnet finetune and api full course video',
    'superb excellent video keep it up',
    'i liked the font you use in the thumbnail is it okay to share its name?',
    'are these numbers even legit or just an exaggerated estimation?'
]

# Tokenize input comments
inputs = tokenizer(comments_list, padding=True, truncation=True, max_length=512, return_tensors="tf")

# Run model inference
outputs = infer(**inputs)
logits = outputs['logits'].numpy()

# Define label mapping (0: Other, 1: Suggestion)
label_mapping = {0: "Other", 1: "Suggestion"}

# Convert logits to labels
predictions = np.argmax(logits, axis=1)  

# Display results
for index, pred in enumerate(predictions):
    print(comments_list[index], ":", label_mapping[pred])

misleading title click bait : Other
kindly post video about claude 35 sonnet finetune and api full course video : Suggestion
superb excellent video keep it up : Other
i liked the font you use in the thumbnail is it okay to share its name? : Other
are these numbers even legit or just an exaggerated estimation? : Other


In [9]:
import os
import zipfile

# Path to your saved model
model_path = "/kaggle/working/bert_model_savedsuggestionmodel"

# Output zip path
zip_output_path = "/kaggle/working/bert_model_savedsuggestionmodel.zip"

# Zip the model directory
with zipfile.ZipFile(zip_output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(model_path):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, os.path.dirname(model_path))
            zipf.write(filepath, arcname)

print(f"✅ Model zipped and ready at: {zip_output_path}")


✅ Model zipped and ready at: /kaggle/working/bert_model_savedsuggestionmodel.zip
